In [1]:
with open('input.txt', 'r', encoding='utf-8') as file:
    text = file.read()

print("length of the dataset in charactes : ", len(text))


length of the dataset in charactes :  1115394


In [2]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [3]:
# here are all the unique characters that occur in the this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [7]:
# create a ampping from characters to integers

stoi = {ch:i for i, ch, in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
encode = lambda s : [stoi[c] for c in s]
decode = lambda l : ''.join([itos[i] for i in l])

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [8]:
# lets now encode the entire text dataset and store it in into a torch.tensor

import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [9]:
# Lets now split up the data into train and validation sets

n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]
print(train_data.shape, val_data.shape)

torch.Size([1003854]) torch.Size([111540])


In [10]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [11]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target is : {target}")

when input is tensor([18]) the target is : 47
when input is tensor([18, 47]) the target is : 56
when input is tensor([18, 47, 56]) the target is : 57
when input is tensor([18, 47, 56, 57]) the target is : 58
when input is tensor([18, 47, 56, 57, 58]) the target is : 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target is : 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target is : 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is : 58


In [12]:
batch_size = 4
block_size = 8
ix = torch.randint(len(data)-block_size, (batch_size,))
# since if you take the last block of data then torch.stack 
# operation will produce out of bounds exception
ix

tensor([ 308062, 1078885,  729185,  328670])

In [14]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data)-block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs: ')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('--------')

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"when input is {context.tolist()} the target is : {target}")


inputs: 
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
--------
when input is [24] the target is : 43
when input is [24, 43] the target is : 58
when input is [24, 43, 58] the target is : 5
when input is [24, 43, 58, 5] the target is : 57
when input is [24, 43, 58, 5, 57] the target is : 1
when input is [24, 43, 58, 5, 57, 1] the target is : 46
when input is [24, 43, 58, 5, 57, 1, 46] the target is : 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target is : 39
when input is [44] the target is : 53
when input is [44, 53] the target is : 56
when input is [44, 53, 56] the target is : 1
when input is [44, 53, 56, 1] the target is : 58
when input is [44

In [16]:
# Bigram langauge model
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(1337)
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding = nn.Embedding(vocab_size, vocab_size)
    def forward(self, idx, targets = None):
        # idx and targets are both [batch_size, block_size]
        logits = self.token_embedding(idx) # [batch_size, block_size, vocab_size]
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C) # 32 * 65
            targets = targets.view(B*T) # 32
            loss = F.cross_entropy(logits, targets) # 32 * 65
        return logits, loss
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            # focus  only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat([idx, idx_next], dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)


In [35]:
m.token_embedding(xb).shape, yb.shape

(torch.Size([4, 8, 65]), torch.Size([4, 8]))

In [37]:
yb.view(-1), yb, logits.shape, yb.view(32).shape

(tensor([43, 58,  5, 57,  1, 46, 43, 39, 53, 56,  1, 58, 46, 39, 58,  1, 58,  1,
         58, 46, 39, 58,  1, 46, 17, 27, 10,  0, 21,  1, 54, 39]),
 tensor([[43, 58,  5, 57,  1, 46, 43, 39],
         [53, 56,  1, 58, 46, 39, 58,  1],
         [58,  1, 58, 46, 39, 58,  1, 46],
         [17, 27, 10,  0, 21,  1, 54, 39]]),
 torch.Size([32, 65]),
 torch.Size([32]))

In [27]:
idx = torch.zeros((1, 1), dtype = torch.long)
idx
print(decode(m.generate(idx,max_new_tokens=100)[0].tolist()))


SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [30]:
logits, loss = m(idx)

In [32]:
logits.shape

torch.Size([1, 1, 65])

In [39]:
F.softmax(logits[:, -1, :])[:, 54]

/var/folders/2g/pfqbqwwn5rdd50z09bnwq59c0000gn/T/ipykernel_38221/2961418082.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  F.softmax(logits[:, -1, :])[:, 54]


tensor([0.1199], grad_fn=<SelectBackward0>)

In [38]:
# create a Pytorch optimizer

optimizer = torch.optim.Adam(m.parameters(), lr=1e-3)


In [39]:
batch_size=32
for steps in range(10000):
    xb, yb = get_batch('train')
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    if steps % 1000 == 0:
        print(f"Step {steps}, loss = {loss.item()}")

Step 0, loss = 4.648484230041504
Step 1000, loss = 3.707036018371582
Step 2000, loss = 3.1244237422943115
Step 3000, loss = 2.7269415855407715
Step 4000, loss = 2.567491292953491
Step 5000, loss = 2.4887678623199463
Step 6000, loss = 2.4732906818389893
Step 7000, loss = 2.414722204208374
Step 8000, loss = 2.433043956756592
Step 9000, loss = 2.451821804046631


In [40]:
print(decode(m.generate(torch.zeros((1, 1), dtype = torch.long),max_new_tokens=500)[0].tolist()))


LA c wo the;
Pancalolinghowhatharean:
QA:

WI ass bowoond;
Fomere d shacenotep.
CI y mbotot swefeseallo br. ave aviasurf my, mayo t ivee iuedrd whar ksth y h bora s be hese, woweee; the! KI 'de, ulseecherd d o blllando;

Whan.
Buspowhald ve!
RIfans picspeserer hee anf,
TOFonk? me ain ckntoty dedo bo'llll st ta d:
ELIS me hurf lal y, ma dus pe athouo
Bin!! Indy; by s afreanoo adicererupa anse tecorro llaus a!
OLengerithesinthengou beal amas trr
TI ar I t, mes, o sar; my w, fredeeyove
THek' merer,


In [41]:
# The mathematical trick in self.attention

torch.manual_seed(1337)
B, T, C = 4, 8, 2
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

In [42]:
# we want x[b, t] = mean_{i<=t} x[b, i] 
xbow = torch.zeros(B, T, C)
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1] # (t, C)
        xbow[b, t] = torch.mean(xprev, 0) # summing over t dimension gives a vector of vocab size 2 here.




In [50]:
x[0]

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

In [51]:
xbow[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [56]:
wei = torch.tril(torch.ones(T, T))
wei = wei / torch.sum(wei, 1, keepdim=True)
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [58]:
xbow2 = wei @ x # T, T multiplied with B, T, C , creates a fake batch dimension for wei, 1, T, T
                # (1 , T,  T) * (B , T , C) --> B, T, C
                # This broadcasts into (B, T, T) 

In [59]:
torch.allclose(xbow, xbow2)

True

In [67]:
# version 3 using softmax

tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim = 1)
xbow3 = wei @ x

torch.allclose(xbow2, xbow3)

True

In [55]:
torch.manual_seed(42)
a = torch.tril(torch.ones((3, 3)))
a = a/torch.sum(a, 1, keepdim=True)
b = torch.randint(0, 10, (3, 2)).float()

c = a @ b

print('a==')
print(a)
print('b==')
print(b)
print('c==')
print(c)


# Based on the above code intuition , lets go back above and implement the xbow with efficient matrix

a==
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
b==
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
c==
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [69]:
# version 4 : self attention
torch.manual_seed(1337)
B, T, C = 4, 8, 32 # batch, time , channels
x = torch.randn(B, T, C) # B, T, C
# Lets see a single Head perform self - attention
head_size = 16
query = nn.Linear(C, head_size)
key = nn.Linear(C, head_size)
value = nn.Linear(C, head_size)

k = key(x) # B, T, 16
q = query(x) # B, T, 16
wei = q @ k.transpose(-2, -1)  # (B, T, head_size)  * (B, head_size, T) --> (B, T, T)
tril = torch.tril(torch.ones(T, T))
wei = wei.masked_fill(tril==0, float("-inf"))
wei = F.softmax(wei, dim = -1)
print(wei.shape)

val = value(x)
out = wei @ val # (B, T, T) * (B, T, head_size) --> B, T, head_size
out.shape


torch.Size([4, 8, 8])


torch.Size([4, 8, 16])

In [75]:
q[0].shape,k.transpose(-2, -1)[0].shape
# One word takes dot product with all of the words and stores as the first row, but by using tril we dont want to take future tokens so 
# we nullify it by using tril mask fill of inf and softmax combination

(torch.Size([8, 16]), torch.Size([16, 8]))